In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
import xgboost as xgb
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_log_error

In [2]:
import warnings

# Ignorar todos os warnings
warnings.filterwarnings("ignore")


In [3]:
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")


In [4]:
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [5]:
train.isna().sum()

id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64

In [6]:
X_train = train.drop(columns=['sales', 'id'])
y_train = train['sales']

In [7]:
X_test = test.drop(columns=['id'])

In [8]:
X_train.dtypes

date           object
store_nbr       int64
family         object
onpromotion     int64
dtype: object

**Separar as variáveis numéricas e categoricas para tratar elas de maneira separada**

In [9]:
numerical = X_train.select_dtypes(include=['int64','float64']).columns
categorical = X_train.select_dtypes(include=['object']).columns

**Com as variáveis separadas podemos tratar cada uma com suas propriedades específicas, sendo que as categóricas tem que ser convertidas para que nosso modelo consiga entender e usá-las.**

Para isso podemos usar a função Pipeline do scikit learn para estruturar melhor cada etapa do tratamento


In [10]:
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

In [11]:
categorical_transformer = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

#### Possíveis Dúvidas

* **SimpleImputer**, é um método em que quando chamado ele calcula a estratégia escolhida (se escolher média, ele calcula a média da coluna), daí caso haja valores NULL na coluna ele irá substituir aqueles valores pela média (caso você escolha média como estratégia). Algumas estratégias são:
    * Média (mean)
    * Mediana (median)
    * Constante especificada (constant)
    * Valor Mais frequente (most_frequent)

* **StandardScaler**, biblioteca que padroniza os dados numéricos, garantindo que todas estejam na mesma escala antes de alimentar o modelo. Padroniza os dados para que tenham, média = 0 e desvio padrão = 1

* **OneHotEncoder**, biblioteca utilizada para converter dados categóricos em dados numéricos, chamados de variáveis dummy. Criando colunas separadas para cada categoria e preenchendo elas com valores binários 0 ou 1.
    * O **handle_unknown = 'ignore'** ignora valores que possam aparecer nos dados de teste que não estavam nos dados de treinamento, assim evitando que gere erros. 

In [12]:
preprocessor = ColumnTransformer(
    transformers = [
        ('num', numerical_transformer, numerical),
        ('cat', categorical_transformer, categorical)
    ]
)

### Possíveis Dúvidas
* **ColumnTransformer**, é a biblioteca responsável por aplicar as tranformações necessárias às colunas que estamos tratando. De acordo com o scikit learn:
    ```python
    ColumnTransformer(transformers = [
        List of Tuples
        ('name', transformers, columns)
    ])
    ```

In [13]:
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', xgb.XGBRegressor(n_estimators = 100, random_state = 42))

])

### Possíveis Dúvidas 
* **preprocessor**, aqui a gente só está definindo que quando chamarmos o model_pipeline ele passará os dados pelo preprocessor que definimos anteriormente.

* **xgb.XGBRegressor**, modelo de regressão baseado em árvores de decisão com boosting, que combina várias árvores de decisão para criar um modelo forte. O XGBRegressor mais especificamente é ótimo para lidar com problemas de regressão, para prever valores contínuos.

In [14]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['store_nbr', 'onpromotion'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['da...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=100, n_jobs=None,
                              num_parallel_tree=None, random_state=42, ...))])

### Possíveis Dúvidas
* **model_pipeline.fit(X_train, y_train)**, usa o X_train e y_train para alimentar nosso pipeline, primeiro passando os valores pelo tratamento do preprocessor, e depois usando o resultado para alimentar o modelo XGBRegressor.

In [15]:
y_valid_pred = model_pipeline.predict(X_valid)

In [16]:
mae = mean_absolute_error(y_valid, y_valid_pred)
mse = mean_squared_error(y_valid, y_valid_pred)
rmse = mean_squared_error(y_valid, y_valid_pred, squared=False)
r2 = r2_score(y_valid, y_valid_pred)

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'R-squared (R²): {r2}')


Mean Absolute Error (MAE): 141.6127126836877
Mean Squared Error (MSE): 206429.9783505723
Root Mean Squared Error (RMSE): 454.3456595485119
R-squared (R²): 0.8339008603068766


In [17]:
y_test_pred = model_pipeline.predict(X_test)

In [18]:
y_test_pred

array([-6.9397032e-01, -6.9397032e-01,  4.9221275e+01, ...,
        1.3448480e+03,  3.2997186e+02,  2.9253798e+01], dtype=float32)

In [19]:
df = pd.DataFrame({
    'Actual': y_valid.astype(int),
    'Predicted': y_valid_pred.astype(int)
})
print(df)

         Actual  Predicted
2483653    2576       2421
2761877       0        135
1217988     357        180
1231938       5         12
528175        0         88
...         ...        ...
2640551       3         25
1201944     258         77
12974       292        221
1136162     738        531
2582492    1412        526

[600178 rows x 2 columns]
